Using QCSubmit to make an OptimizationDataset from the starting PDBS and JSON.

In [1]:
from qcsubmit.datasets import OptimizationDataset
from qcsubmit.serializers import deserialize
from qcsubmit.constraints import Constraints
from qcsubmit.factories import OptimizationDatasetFactory
from openforcefield.topology import Molecule
from openforcefield.utils.toolkits import RDKitToolkitWrapper

In [2]:
# make the dataset and set the name and info
dataset = OptimizationDataset(dataset_name="OpenFF-Disaccharides-v1.0", dataset_tagline="Constrained Disaccharide Optimizations", description="Constrained Disaccharide Optimizations for force field development generated by mdgx.")

In [3]:
dataset.metadata.submitter = "David Cerutti"

In [4]:
dataset.metadata.long_description_url = "https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2020-07-28-OpenFF-Disaccharide-Optimization-Set"

In [5]:
dataset

OptimizationDataset(qc_specifications={'default': QCSpec(method='B3LYP-D3BJ', basis='DZVP', program='psi4', spec_name='default', spec_description='Standard OpenFF optimization quantum chemistry specification.', store_wavefunction=<WavefunctionProtocolEnum.none: 'none'>)}, dataset_name='OpenFF-Disaccharides-v1.0', dataset_tagline='Constrained Disaccharide Optimizations', dataset_type='OptimizationDataset', maxiter=200, driver=<DriverEnum.gradient: 'gradient'>, scf_properties=['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices'], priority='normal', description='Constrained Disaccharide Optimizations for force field development generated by mdgx.', dataset_tags=['openff'], compute_tag='openff', metadata=Metadata(submitter='David Cerutti', creation_date=datetime.date(2020, 9, 1), collection_type='OptimizationDataset', dataset_name='OpenFF-Disaccharides-v1.0', short_description='Constrained Disaccharide Optimizations', long_description_url=HttpUrl('https://github.com/openforcef

Now lets look at how we could make a dataset entry using the input JSON which contains the conectivity, atoms, constraints and coordinates in bohr.
Here all of the data is in the First1000 folder which contains a folder for each molecule which has the starting geometry in JSON format. 

## Steps:
- Extract the molecule information from a JSON sybmols, conectivity, coords
- Build an openFF molecule
- Write out a PDB file
- Read the file back to triger seterochem and bonding checks in openeye
- add all geometries
- add to the dataset

It would probably be better to supply the input PDB files to ensure correct CMILES.



In [6]:
# lets make a function to process each molecule
import os
from simtk import unit
import numpy as np

def extract_molecule_data(molecule_data):
    """
    Given the molecule data dict extract only the inportant bits.
    Here we assume there is one conformation per JSON.
    """
    name = list(molecule_data["dataset"].keys())[0]
    molecule = molecule_data["dataset"][name]["initial_molecules"][0]
    return name, molecule

def extract_constraints(molecule_data, name):
    """
    Given the molecule data dict extract and build the constraints.
    """
    molecule_constraints = molecule_data["dataset"][name]["keywords"]["constraints"]["freeze"]
    constraints = Constraints()
    for constraint in molecule_constraints:
        constraints.add_freeze_constraint(constraint_type=constraint["type"], indices=constraint["indices"])
    print(constraints)
    return constraints
    

def build_off_mol(name, molecule_data):
    """
    Build an off molecule from the name and input data.
    """
    atom_map = {"H": 1, "C": 6, "O": 8, "N":7}
    off_mol = Molecule()
    off_mol.name = name
    # add atoms
    for atom in molecule_data["symbols"]:
        off_mol.add_atom(atom_map[atom], 0, False)
    # add bonds
    for bond in molecule_data["connectivity"]:
        off_mol.add_bond(*bond, False)
    # add geometry
    conformer = unit.Quantity(np.array(molecule_data["geometry"]).reshape(-1, 3), unit.bohr)
    off_mol.add_conformer(conformer)
    
    return off_mol
    

def add_molecules_to_dataset(dataset, folder_path):
    """
    For the given folder path process the input molecules into the dataset, note molecules may have different constraints
    """
    print(folder_path)
    molecules_names = []
    factory = OptimizationDatasetFactory()
    
    for file in os.listdir(folder_path):
        if file.endswith(".json"):
            # get the input data
            print(file)
            molecule_data = deserialize(os.path.join(folder_path, file))
            name, mol_data = extract_molecule_data(molecule_data)
            constraints = extract_constraints(molecule_data, name)
            molecules_names.append(name)
            # now add to the dataset as a unique entry
            file_name = os.path.join(folder_path, f"{name}.pdb")
            molecule = build_off_mol(name, mol_data)
            molecule.to_file(file_name, "pdb", toolkit_registry=RDKitToolkitWrapper())
            # now read back
            final_mol = Molecule.from_file(file_name)
            attributes = factory.create_cmiles_metadata(final_mol)
            assert name not in dataset.dataset
            dataset.add_molecule(index=final_mol.name, molecule=final_mol, attributes=attributes, constraints=constraints)
        
#     # get the molecule data as pdb
#     molecule = molecules.pop()
#     file_name = os.path.join(folder_path, f"{molecule.name}.pdb")
#     molecule.to_file(file_name, "pdb", toolkit_registry=RDKitToolkitWrapper())
#     # now read back
#     final_mol = Molecule.from_file(file_name)
#     # get the CMILES
#     attributes = factory.create_cmiles_metadata(final_mol)
#     # now add all conformers
#     for conformer in molecules:
#         final_mol.add_conformer(conformer._conformers[0])
#     dataset.add_molecule(index=final_mol.name, molecule=final_mol, attributes=attributes, constraints=constraints)

            
            
#         elif file.endswith(".json") and "constraints" not in file:
#             # this has the coords and name
#             mol_data = deserialize(os.path.join(folder_path, file))
            
    # now we should have everything we need
    # make the new geometry and get the name, only one mol per json
#     for mol_json in mol_data["dataset"].values():
#         index = mol_json["index"]
#         conformer = unit.Quantity(np.array(mol_json["initial_molecules"][0]["geometry"]).reshape((mol.n_atoms, 3)), unit.angstrom)
#         mol.add_conformer(conformer)
#         break
#     assert mol.n_conformers == 1
#     # now add the molecule
#     attributes = factory.create_cmiles_metadata(mol)
#     dataset.add_molecule(index=index, molecule=mol, attributes=attributes, constraints=constraint)
    

In [7]:
# now for each folder lets add the molecule to the dataset
for folder in os.listdir("First1000"):
    if "ipynb" not in folder and os.path.isdir(os.path.join("First1000", folder)):
        add_molecules_to_dataset(dataset, os.path.join("First1000", folder))
    
dataset

First1000/All_Sor
LAllpb1-6DSorfb2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(17, 16, 14, 18)), DihedralConstraint(type='dihedral', indices=(14, 18, 20, 21)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
LAllpa1-3DSorfa2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(15, 17, 20, 21)), DihedralConstraint(type='dihedral', indices=(15, 13, 23, 24)), DihedralConstraint(type='dihedral', indices=(29, 31, 34, 35)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(43, 42, 40, 44)), DihedralConstraint(type='dihedral', indices=(40, 44, 46, 47))] set=[]
LAllpb1-4DSorfa2-OME.jso

DAllpb1-3DSorfa2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(15, 17, 20, 21)), DihedralConstraint(type='dihedral', indices=(15, 13, 23, 24)), DihedralConstraint(type='dihedral', indices=(29, 31, 34, 35)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(43, 42, 40, 44)), DihedralConstraint(type='dihedral', indices=(40, 44, 46, 47))] set=[]
DAllpa1-6DSorfb2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(17, 16, 14, 18)), DihedralConstraint(type='dihedral', indices=(14, 18, 20, 21)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
DAllpb1-4LSorfb2-OME.json
freeze=[Dihedral

LAllpb1-3LSorfb2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(15, 17, 20, 21)), DihedralConstraint(type='dihedral', indices=(15, 13, 23, 24)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
LAllpa1-4LSorfb2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(14, 13, 11, 15)), DihedralConstraint(type='dihedral', indices=(17, 19, 22, 23)), DihedralConstraint(type='dihedral', indices=(29, 31, 34, 35)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(43, 42, 40, 44)), DihedralConstraint(type='dihedral', indices=(40, 44, 46, 47))] set=[]
First1000/All_Psi
DAllpb1-3DPsifa2-OME.jso

LAllpb1-6DPsifb2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(17, 16, 14, 18)), DihedralConstraint(type='dihedral', indices=(14, 18, 20, 21)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
LAllpa1-3DPsifa2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(15, 17, 20, 21)), DihedralConstraint(type='dihedral', indices=(15, 13, 23, 24)), DihedralConstraint(type='dihedral', indices=(29, 31, 34, 35)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(43, 42, 40, 44)), DihedralConstraint(type='dihedral', indices=(40, 44, 46, 47))] set=[]
LAllpa1-4LPsifb2-OME.json
freeze=[Dihedral

DAllpb1-6LPsifa2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(17, 16, 14, 18)), DihedralConstraint(type='dihedral', indices=(14, 18, 20, 21)), DihedralConstraint(type='dihedral', indices=(29, 31, 34, 35)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(43, 42, 40, 44)), DihedralConstraint(type='dihedral', indices=(40, 44, 46, 47))] set=[]
DAllpb1-4LPsifb2-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(5, 6, 9, 10)), DihedralConstraint(type='dihedral', indices=(14, 13, 11, 15)), DihedralConstraint(type='dihedral', indices=(17, 19, 22, 23)), DihedralConstraint(type='dihedral', indices=(29, 31, 34, 35)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(43, 42, 40, 44)), DihedralConstraint(type='dihedral', indices=(40, 44, 46, 47))] set=[]
First1000/All_Xyl
LAllpb1-3LXylfa1-OME.jso

...

DAllpb1-3LRibfb1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(10, 9, 7, 11)), DihedralConstraint(type='dihedral', indices=(13, 15, 18, 19)), DihedralConstraint(type='dihedral', indices=(25, 27, 30, 31)), DihedralConstraint(type='dihedral', indices=(35, 34, 32, 36)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(36, 40, 42, 43))] set=[]
LAllpa1-2DRibfb1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(8, 10, 13, 14)), DihedralConstraint(type='dihedral', indices=(18, 17, 15, 19)), DihedralConstraint(type='dihedral', indices=(25, 27, 30, 31)), DihedralConstraint(type='dihedral', indices=(35, 34, 32, 36)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(36, 40, 42, 43))] set=[]
DAllpa1-3LRibfb1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(10, 9, 7, 11)), DihedralConstraint(type='dihedral', indices=(13, 15, 18, 19)), Dihedra

DAllpa1-2DGulpa1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(13, 15, 18, 19)), DihedralConstraint(type='dihedral', indices=(13, 11, 21, 22)), DihedralConstraint(type='dihedral', indices=(11, 9, 23, 24)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
LAllpa1-4LGulpa1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(8, 10, 13, 14)), DihedralConstraint(type='dihedral', indices=(20, 19, 17, 21)), DihedralConstraint(type='dihedral', indices=(17, 21, 23, 24)), DihedralConstraint(type='dihedral', indices=(29, 31, 34, 35)), DihedralConstraint(type='dihedral', indices=(39, 38, 36, 40)), DihedralConstraint(type='dihedral', indices=(43, 42, 40, 44)), DihedralConstraint(type='dihedral', indices=(40, 44, 46, 47))] set=[]
LAllpb1-3LGulpa1-OME.json
freeze=[Dihe

DAllpa1-3LGulpa1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(8, 10, 13, 14)), DihedralConstraint(type='dihedral', indices=(18, 17, 15, 19)), DihedralConstraint(type='dihedral', indices=(19, 21, 23, 24)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
LAllpb1-2DGulpa1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(13, 15, 18, 19)), DihedralConstraint(type='dihedral', indices=(13, 11, 21, 22)), DihedralConstraint(type='dihedral', indices=(11, 9, 23, 24)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
DAllpb1-4LGulpa1-OME.json
freeze=[Dihe

LAllpb1-3LGulpb1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(10, 9, 7, 11)), DihedralConstraint(type='dihedral', indices=(15, 17, 20, 21)), DihedralConstraint(type='dihedral', indices=(15, 13, 23, 24)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
DAllpa1-2DGulpb1-OME.json
freeze=[DihedralConstraint(type='dihedral', indices=(8, 10, 13, 14)), DihedralConstraint(type='dihedral', indices=(18, 17, 15, 19)), DihedralConstraint(type='dihedral', indices=(22, 21, 19, 23)), DihedralConstraint(type='dihedral', indices=(34, 36, 39, 40)), DihedralConstraint(type='dihedral', indices=(34, 32, 42, 43)), DihedralConstraint(type='dihedral', indices=(32, 30, 44, 45)), DihedralConstraint(type='dihedral', indices=(30, 28, 46, 47))] set=[]
LAllpa1-4LGulpb1-OME.json
freeze=[Dihed

OptimizationDataset(qc_specifications={'default': QCSpec(method='B3LYP-D3BJ', basis='DZVP', program='psi4', spec_name='default', spec_description='Standard OpenFF optimization quantum chemistry specification.', store_wavefunction=<WavefunctionProtocolEnum.none: 'none'>)}, dataset_name='OpenFF-Disaccharides-v1.0', dataset_tagline='Constrained Disaccharide Optimizations', dataset_type='OptimizationDataset', maxiter=200, driver=<DriverEnum.gradient: 'gradient'>, scf_properties=['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices'], priority='normal', description='Constrained Disaccharide Optimizations for force field development generated by mdgx.', dataset_tags=['openff'], compute_tag='openff', metadata=Metadata(submitter='David Cerutti', creation_date=datetime.date(2020, 9, 1), collection_type='OptimizationDataset', dataset_name='OpenFF-Disaccharides-v1.0', short_description='Constrained Disaccharide Optimizations', long_description_url=HttpUrl('https://github.com/openforcef

In [8]:
dataset.n_molecules

976

In [9]:
dataset.n_records

976

In [10]:
dataset.metadata.elements

{'C', 'H', 'O'}

In [11]:
dataset.dataset.keys()

dict_keys(['OME_6BU_0nB', 'OME_3BD_0nA', 'OME_4BD_0nB', 'OME_4bD_0nB', 'OME_3bD_0nA', 'OME_6bU_0nB', 'OME_4bD_0nA', 'OME_3bD_0nB', 'OME_6bU_0nA', 'OME_6BU_0nA', 'OME_3BD_0nB', 'OME_4BD_0nA', 'OME_4BD_0NB', 'OME_3BD_0NA', 'OME_6BU_0NB', 'OME_6bU_0NB', 'OME_3bD_0NA', 'OME_4bD_0NB', 'OME_6bU_0NA', 'OME_3bD_0NB', 'OME_4bD_0NA', 'OME_4BD_0NA', 'OME_3BD_0NB', 'OME_6BU_0NA', 'OME_4bU_0NB', 'OME_3bU_0NA', 'OME_6bD_0NB', 'OME_6BD_0NB', 'OME_3BU_0NA', 'OME_4BU_0NB', 'OME_6BD_0NA', 'OME_3BU_0NB', 'OME_4BU_0NA', 'OME_4bU_0NA', 'OME_3bU_0NB', 'OME_6bD_0NA', 'OME_6bD_0nB', 'OME_3bU_0nA', 'OME_4bU_0nB', 'OME_4BU_0nB', 'OME_3BU_0nA', 'OME_6BD_0nB', 'OME_4BU_0nA', 'OME_3BU_0nB', 'OME_6BD_0nA', 'OME_6bD_0nA', 'OME_3bU_0nB', 'OME_4bU_0nA', 'OME_3PD_0NB', 'OME_6PU_0NA', 'OME_4PD_0NA', 'OME_4pD_0NA', 'OME_6pU_0NA', 'OME_3pD_0NB', 'OME_4pD_0NB', 'OME_6pU_0NB', 'OME_3pD_0NA', 'OME_3PD_0NA', 'OME_6PU_0NB', 'OME_4PD_0NB', 'OME_4PD_0nA', 'OME_6PU_0nA', 'OME_3PD_0nB', 'OME_3pD_0nB', 'OME_6pU_0nA', 'OME_4pD_0nA',

In [12]:
dataset.dataset["OME_3nB_0nA"].off_molecule

NGLWidget()

In [13]:
dataset.dataset["OME_6nA_0nB"].off_molecule

NGLWidget()

In [14]:
dataset.export_dataset("dataset.json")

In [15]:
dataset.visualize("Disaccharide.pdf")

In [16]:
dataset.molecules_to_file("Disaccharide.smi", "smi")

In [15]:
mol = Molecule.from_mapped_smiles("[H:16][C@@:15]1([C@@:11]([C@:9]([O:8][C@@:6]([C@@:19]1([H:20])[O:21][H:22])([H:7])[O:5][C:2]([H:1])([H:3])[H:4])([H:10])[C:23]([H:24])([H:25])[O:26][C@@:27]2([C@@:29]([C@@:31]([C@@:33]([C@:35]([O:42]2)([H:36])[C:37]([H:38])([H:39])[O:40][H:41])([H:34])[O:43][H:44])([H:32])[O:45][H:46])([H:30])[O:47][H:48])[H:28])([H:12])[O:13][H:14])[O:17][H:18]")
mol.generate_conformers(n_conformers=1)

In [16]:
mol

NGLWidget()

In [17]:
entry = dataset.get_molecule_entry(mol)

KeyboardInterrupt: 

In [ ]:
entry

In [18]:
molecules= list(dataset.molecules)

In [19]:
molecules[6]

NGLWidget()

In [20]:
for molecule in dataset.molecules:
    print(dataset.get_molecule_entry(molecule))

['OME_6BU_0nB']
['OME_3BD_0nA']
['OME_4BD_0nB']
['OME_4bD_0nB']
['OME_3bD_0nA']
['OME_6bU_0nB']
['OME_4bD_0nA']
['OME_3bD_0nB']


KeyboardInterrupt: 